In [1]:
# Import packages
import os
import time
import tensorflow as tf
from google.colab import drive

# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

# Import Remaining Packages (Post-Install)
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import col, count


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [456 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [979 kB]
Get:13 http://archive.ubuntu

In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkML").getOrCreate()

In [3]:
#connect to google drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# double check to make sure your pathway to the csv's is the same!!!!

#read in the csv data from google drive
spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/train_timeseries.csv')
timeseries_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/train_timeseries.csv'), sep=",", header=True, inferSchema=True)

spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/test_timeseries.csv')
test_timeseries_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/test_timeseries.csv'), sep=",", header=True, inferSchema=True)

# spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/validation_timeseries.csv')
# validation_timeseries_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/validation_timeseries.csv'), sep=",", header=True, inferSchema=True)

spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/soil_data.csv')
soil_data_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/soil_data.csv'), sep=",", header=True, inferSchema=True)

In [5]:
# # Read in CSVs into Data Frames
# spark.sparkContext.addFile('soil_data.csv')
# soil_data_df = spark.read.csv(SparkFiles.get('soil_data.csv'), sep=",", header=True, inferSchema=True)

# spark.sparkContext.addFile('test_timeseries.csv')
# test_timeseries_df = spark.read.csv(SparkFiles.get('test_timeseries.csv'), sep=",", header=True, inferSchema=True)

In [6]:
soil_data_df.show()

+----+---------+----------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+-----------------+-----------------+----------------+-----------------+-------------------+-------------------+---+---+---+---+---+---+---+
|fips|      lat|       lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|         URB_LAND|         GRS_LAND|        FOR_LAND|      CULTRF_LAND|        CULTIR_LAND|          CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|
+----+---------+----------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+-----------------+-----------------+----------------+-----------------+-------------------+-------------------+---+---+---+---+---+---+---+
|1001|32.536382| -86.64449|       63|0.0419|0.2788|0.2984|

In [7]:
timeseries_df.show()

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|
+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+
|1001|2000-01-01|   0.22|100.51| 9.65|14.74| 13.51| 13.51|  20.96|  11.46|      9.5|14.65|  2.2|     2.94|     1.49|       1.46| 4.85|     6.04|     3.23|       2.81| null|
|1001|2000-01-02|    0.2|100.55|10.42|16.69| 14.71| 14.71|   22.8|  12.61|    10.18| 16.6| 2.52|     3.43|     1.83|        1.6| 5.33|     6.13|     3.72|       2.41| null|
|1001|2000-01-03|   3.65|100.15|11.76|18.49| 16.52| 16.52|  22.73|  15.32|     7.41|18.41| 4.03|     5.33|     2.66|       2.67| 7.53| 

In [8]:
# filled_test_timeseries_df = test_timeseries_df.fillna({'score':5})
#filled_test_timeseries_df = test_timeseries_df

from pyspark.sql import functions as F


modified_timeseries_df = timeseries_df.withColumn('score', F.col('score') + 1)
filled_timeseries_df = modified_timeseries_df.fillna({'score':0}) # Fill null values with 0


In [9]:
joined_df = filled_timeseries_df.join(soil_data_df, on='fips', how='left')
# Round Score column
from pyspark.sql.functions import round, col
from pyspark.sql.types import IntegerType

# Round the values in the column
# joined_df = joined_df.withColumn("score", round(col("score")))

# Cast the rounded values to integer
#joined_df = joined_df.withColumn("score", col("score").cast("integer"))

joined_df.show()

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+---------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|      lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|
+----+----------+-------+------+-----+-----+------+------+--

In [10]:
#joined_df = joined_df.withColumn("score", round(joined_df["score"]).cast(IntegerType()))

In [11]:
# Count of Unique Locations in Soil Data
soil_locations = soil_data_df.groupBy('fips').agg(count("*").alias("count"))
soil_locations.count()
print(f"Soil Data contains {soil_locations.count()} unique locations")

# Count of Unique Locations in Test Timeseries
timeseries_locations = timeseries_df.groupBy('fips').agg(count("*").alias("count"))
timeseries_locations.count()
print(f"Timeseries contains {timeseries_locations.count()} unique locations")
print("---")

# Print shapes of dataframes
print(f"Soil Data contains {soil_data_df.count()} rows and {len(soil_data_df.columns)} columns")
print(f"Timeseries contains {timeseries_df.count()} rows and {len(timeseries_df.columns)} columns")
print(f"Joined contains {joined_df.count()} rows and {len(joined_df.columns)} columns")

Soil Data contains 3109 unique locations
Timeseries contains 3108 unique locations
---
Soil Data contains 3109 rows and 32 columns
Timeseries contains 19300680 rows and 21 columns
Joined contains 19300680 rows and 52 columns


In [12]:
joined_df.printSchema()

root
 |-- fips: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- PRECTOT: double (nullable = true)
 |-- PS: double (nullable = true)
 |-- QV2M: double (nullable = true)
 |-- T2M: double (nullable = true)
 |-- T2MDEW: double (nullable = true)
 |-- T2MWET: double (nullable = true)
 |-- T2M_MAX: double (nullable = true)
 |-- T2M_MIN: double (nullable = true)
 |-- T2M_RANGE: double (nullable = true)
 |-- TS: double (nullable = true)
 |-- WS10M: double (nullable = true)
 |-- WS10M_MAX: double (nullable = true)
 |-- WS10M_MIN: double (nullable = true)
 |-- WS10M_RANGE: double (nullable = true)
 |-- WS50M: double (nullable = true)
 |-- WS50M_MAX: double (nullable = true)
 |-- WS50M_MIN: double (nullable = true)
 |-- WS50M_RANGE: double (nullable = true)
 |-- score: double (nullable = false)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- elevation: integer (nullable = true)
 |-- slope1: double (nullable = true)
 |-- slope2: double (nullable = true)
 

In [13]:
from pyspark.sql import functions as F
joined_df_scores = joined_df.groupBy('score').agg(count("*").alias("count")).orderBy(F.desc("count"))
joined_df_scores.show()

+------------------+--------+
|             score|   count|
+------------------+--------+
|               0.0|16543884|
|               1.0| 1480827|
|               2.0|  219135|
|               3.0|  123789|
|               4.0|   82801|
|               5.0|   45841|
|               6.0|   21806|
|            1.0001|    1037|
|            1.0002|     742|
|1.9998999999999998|     681|
|            1.0003|     642|
|            2.9999|     487|
|            1.0004|     486|
|            1.0005|     478|
|            1.0006|     459|
|2.0000999999999998|     455|
|            1.9998|     454|
|            1.0007|     409|
|            1.9996|     383|
|            1.9997|     353|
+------------------+--------+
only showing top 20 rows



In [14]:
summary_df = joined_df.summary("min", "max")

# Print the results
summary_df.show()

+-------+-----+-------+------+-----+------+------+------+-------+-------+---------+------+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+-----------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+--------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+
|summary| fips|PRECTOT|    PS| QV2M|   T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|    TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|        lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|
+-------+-----+-------+------+-----+------+------+------+-------+-------+-

In [15]:
# Columns that we do not want to include as features
exclude_columns = ['score', 'lat', 'lon', 'fips', 'date']

# Set feature columns (excluding non-feature columns)
feature_columns = [col_name for col_name in joined_df.columns if col_name not in exclude_columns]
feature_columns

['PRECTOT',
 'PS',
 'QV2M',
 'T2M',
 'T2MDEW',
 'T2MWET',
 'T2M_MAX',
 'T2M_MIN',
 'T2M_RANGE',
 'TS',
 'WS10M',
 'WS10M_MAX',
 'WS10M_MIN',
 'WS10M_RANGE',
 'WS50M',
 'WS50M_MAX',
 'WS50M_MIN',
 'WS50M_RANGE',
 'elevation',
 'slope1',
 'slope2',
 'slope3',
 'slope4',
 'slope5',
 'slope6',
 'slope7',
 'slope8',
 'aspectN',
 'aspectE',
 'aspectS',
 'aspectW',
 'aspectUnknown',
 'WAT_LAND',
 'NVG_LAND',
 'URB_LAND',
 'GRS_LAND',
 'FOR_LAND',
 'CULTRF_LAND',
 'CULTIR_LAND',
 'CULT_LAND',
 'SQ1',
 'SQ2',
 'SQ3',
 'SQ4',
 'SQ5',
 'SQ6',
 'SQ7']

In [16]:
# Create the VectorAssembler instance to combine feature columns into a single dense vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembler

VectorAssembler_dee94ed6a6a5

In [17]:
# Apply the VectorAssembler to create the dense vector column
df_with_features = assembler.transform(joined_df)
df_with_features.show()

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+---------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+--------------------+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|      lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|            features|
+----+----------+-

In [18]:
# Apply feature scaling using StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
scaler

StandardScaler_7f0de230380a

In [19]:
# Fit the StandardScaler to compute mean and standard deviation
scaler_model = scaler.fit(df_with_features)
scaler_model

StandardScalerModel: uid=StandardScaler_7f0de230380a, numFeatures=47, withMean=true, withStd=true

In [20]:
# Transform the DataFrame to get scaled feature vectors
df_with_scaled_features = scaler_model.transform(df_with_features)
df_with_scaled_features.show()

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+---------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+--------------------+--------------------+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|      lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|            feature

In [21]:
df_with_scaled_features.show()

+----+----------+-------+------+-----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+---------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+--------------------+--------------------+
|fips|      date|PRECTOT|    PS| QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|      lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|            feature

In [22]:
# Split the preprocessed data into a training and testing dataset
train_data, test_data = df_with_scaled_features.randomSplit([0.8, 0.2], seed=42)

In [23]:
#unique_values_list = df_with_scaled_features.select('score').distinct().rdd.flatMap(lambda x: x).collect()
#unique_values_list[0]+1

In [24]:
#df_with_scaled_features = df_with_scaled_features.withColumnRenamed("score", "score1")

In [25]:
#df_with_scaled_features = df_with_scaled_features.withColumn("label", df["label"].cast(IntegerType()))

In [26]:
from pyspark.ml.regression import LinearRegression

# Define the model
lr = LinearRegression(featuresCol='scaled_features', labelCol='score')

# Train the model
lr_model = lr.fit(train_data)

# Make predictions
predictions = lr_model.transform(test_data)

# Evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="score", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = " + str(rmse))

Root Mean Squared Error (RMSE) on test data = 0.7841389830084733


In [28]:
p_values = lr_model.summary.pValues
for i in range(len(feature_columns)):
    print(f"P-Value of feature {feature_columns[i]}: {p_values[i]}")




Py4JJavaError: ignored